In [ ]:
MODEL_PATH="/home/jan/bsc/best-hlu3ln61"
TXT_PATH="/home/jan/bsc/cataccc"
OUTPUT_DIR="/home/jan/bsc/predictions_anns"

In [ ]:
from transformers import pipeline, AutoModelForTokenClassification, AutoTokenizer
import os
import pandas as pd
import csv

pipe = pipeline("token-classification", MODEL_PATH, aggregation_strategy="first")

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH)

In [ ]:
sentence_doc_idx = []
samples = []
for doc_idx, filename in enumerate([filename for filename in sorted(os.listdir(TXT_PATH)) if filename.endswith(".conll")]):
#filename = "/home/jan/bsc/cataccc/cc_ca39.conll"
    with open(os.path.join(TXT_PATH, filename), 'r') as file:
        doc_sentences = []
        sentence = []
        line = file.readline()
        while line:
            if line == '\n':
                doc_sentences.append(' '.join(sentence))
                sentence = []
            else:
                token = line.split('\t')[3].strip()
                sentence.append(token)
            line = file.readline()
        if sentence:
            doc_sentences.append(' '.join(sentence))
    sentence_doc_idx.extend([doc_idx]*len(doc_sentences))
    samples.extend(doc_sentences)

In [ ]:
if os.path.exists(OUTPUT_DIR) and len([ann_file for ann_file in os.listdir(OUTPUT_DIR) if ann_file.endswith('.ann')]) > 0:
    raise Exception("Output directory already has annotations")
else:
    os.makedirs(OUTPUT_DIR, exist_ok=True)

In [ ]:
pipe(' '.join(["su", "metástasis", '(', 'cáncer', 'de', 'pulmón', ')', ';', 'conllevó','a','COVID', '.']))

In [ ]:
pipe("su metástasis (cáncer de pulmón); conllevó a COVID.")

In [ ]:
pipe("su metástasis ( cáncer de pulmón ) ; conllevó a COVID .")

In [ ]:
[filename for filename in sorted(os.listdir(TXT_PATH)) if filename.endswith(".conll")][0]

In [ ]:
sentence_doc_idx[2]

In [ ]:
%%time
sentences_entities = pipe(samples[:10])
sentences_entities

In [ ]:
for sentence_entities in sentences_entities:
    

In [ ]:
filename = "/home/jan/bsc/distemist-es/test/es-S1575-06202010000200004-1.txt"
with open(os.path.join(TXT_PATH, filename)) as file:
    text = file.read()
preds = pipe(text)
if not preds:
    open(os.path.join(OUTPUT_DIR, os.path.basename(filename).replace('.txt', '.ann')), 'w+').close()
    pass


In [ ]:
df = pd.DataFrame.from_records(preds)
# Remove leading whitespace for starting tokens
df['word'] = df['word'].apply(lambda text: text[1:] if text[0] == " " else text)
#df['word'] = df['word'].str.replace(r'^[\s.,\n]+|[\s.,\n]+$', '') # we would need to adjust the offsets (by doing leading and trailing separately)
df['id'] = [f"T{id+1}" for id in df.index]
df['label_offsets'] = df['entity_group'] + ' ' + df['start'].astype(str) + ' ' + df['end'].astype(str)
df = df.drop(['entity_group', 'start', 'end', 'score'], axis=1)
df = df[['id', 'label_offsets', 'word']]

In [ ]:
df

In [ ]:
try:
    df.to_csv(os.path.join(OUTPUT_DIR, os.path.basename(filename).replace('.txt', '.ann')), quoting=csv.QUOTE_NONE, sep="\t", index=False, header=False)
except Exception as e:
    print(df['word'])
    pass

In [ ]:
%%time
for filename in [filename for filename in os.listdir(TXT_PATH) if filename.endswith(".txt")]:
    if os.path.exists(os.path.join(OUTPUT_DIR, os.path.basename(filename).replace('.txt', '.ann'))):
        continue
    with open(os.path.join(TXT_PATH, filename)) as file:
        text = file.read()
    preds = pipe(text)
    if not preds:
        open(os.path.join(OUTPUT_DIR, os.path.basename(filename).replace('.txt', '.ann')), 'w+').close()
        continue
    df = pd.DataFrame.from_records(preds)
    # Remove leading whitespace for starting tokens
    df['word'] = df['word'].apply(lambda text: text[1:] if text[0] == " " else text)
    df['id'] = [f"T{id+1}" for id in df.index]
    df['label_offsets'] = df['entity_group'] + ' ' + df['start'].astype(str) + ' ' + df['end'].astype(str)
    df = df.drop(['entity_group', 'start', 'end', 'score'], axis=1)
    df = df[['id', 'label_offsets', 'word']]
    try:
        df.to_csv(os.path.join(OUTPUT_DIR, os.path.basename(filename).replace('.txt', '.ann')), quoting=csv.QUOTE_NONE, sep="\t", index=False, header=False)
    except Exception as e:
        print(df['word'])
        continue

In [ ]:
# Process each document through pipe
# Save result as dataframe
# Dataframe to .ann
# Without scores boolean
# aggregation strategy